# Measure the Stiffness Matrix

This notebook is used to measure the stiffness matrix of M2 mirror or surrogate.
The stiffness matrix is defined to be a 78 x 78 matrix that has the relationship between the actuator step and force.
The procedure is to move the actuator under the open-loop control with a series of step movements.

This test would need to use the telemetry binary file to support the data analysis to have the enough force data resolution.
The telemetry data sent from the cell controller to CSC is truncated to save the bandwidth.
Therefore, the data in EFD would not be able to evaluate the stiffness matrix.

## Import Modules

In [ ]:
import asyncio
from datetime import datetime
from lsst.ts import salobj
from lsst.ts.m2com import NUM_ACTUATOR

## Declaration of User-defined Functions

In [ ]:
async def run_test(   
    csc: salobj.Remote,
    max_step: int,
    num_times: int,
    actuators: list[int] | None = None,
) -> None:
    """Run the test.

    Parameters
    ----------
    csc : `salobj.Remote`
        M2 CSC.
    max_step : `int`
        Maximum step to move in each control cycle.
    num_times : `int`
        Number of the repeating times.
    actuators : `list` or None, optional
        0-based actuators. (the default is None)
    """

    if actuators is None:
        actuators = range(NUM_ACTUATOR)

    steps = generate_steps(max_step, num_times)
    for actuator in actuators:
        print(f"Run the {actuator=}")
        await move_actuators(csc, actuator, steps)

In [ ]:
async def move_actuators(
    csc: salobj.Remote,
    actuator: int,
    steps: list[int],
    sleep_time: float = 0.5,
) -> None:
    """Move the actuators.

    Parameters
    ----------
    csc : `salobj.Remote`
        M2 CSC.
    actuator : `int`
        0-based actuator ID to move.
    steps : `list` [`int`]
        Actuator steps.
    sleep_time : `float`, optional
        Sleep time between the movements. (the default is 0.5)
    """

    for step in steps:
        if step != 0:
            await csc.cmd_moveActuator.set_start(actuator=actuator, step=step)

        await asyncio.sleep(sleep_time)

In [ ]:
def generate_steps(max_step: int, num_times: int) -> list[int]:
    """Generate the actuator steps.

    Parameters
    ----------
    max_step : `int`
        Maximum step to move in each control cycle. This value should be <= 30.
    num_times : `int`
        Number of repeating times.

    Returns
    -------
    `list` [`int`]
        Actuator steps.
    """

    steps_forward = list()
    for step in range(max_step + 1):
        for _ in range(num_times):
            steps_forward.append(step)

    steps_backward = steps_forward.copy()
    steps_backward.reverse()

    steps_1 = steps_forward + steps_backward + negate_list(steps_forward) + negate_list(steps_backward)

    steps_2 = steps_1.copy()
    steps_2.reverse()

    return steps_1 + steps_2

In [ ]:
def negate_list(values: list) -> list:
    """Negate the list.

    Parameters
    ----------
    values `list`
        Values to negate.
    """
    return [-value for value in values]

## Prepare the M2 CSC and Put to Enabled state

In [ ]:
domain = salobj.Domain()
m2 = salobj.Remote(domain, "MTM2")
await m2.start_task
await m2.cmd_setLogLevel.set_start(level=10)

In [ ]:
state_m2 = m2.evt_summaryState.get()
if state_m2.summaryState != salobj.State.ENABLED:
    await salobj.set_summary_state(m2, salobj.State.ENABLED, timeout=60)

## Measure the Stiffness Matrix

In [ ]:
time_start = datetime.now()
print(f"UTC time to is {time_start} now.")

In [ ]:
# Trinsition to the open-loop control first
await m2.cmd_switchForceBalanceSystem.set_start(status=False)

In [ ]:
# Move the actuators in the open-loop control
max_step = 12
num_times = 2

await run_test(m2, max_step, num_times, actuators=None)

In [ ]:
# Trinsition back to the closed-loop control
await m2.cmd_switchForceBalanceSystem.set_start(status=True)